In [102]:
from pulp import *
import pandas as pd
import numpy as np
import math

initial_loading = pd.read_csv('initial_loading.csv')
product_specs = pd.read_csv('product_specs.csv')
tam_margins = pd.read_csv('tam_and_margin.csv')
workstation_metrics = pd.read_csv('workstation_metrics.csv').T.fillna(0)

workstation_metrics.columns = workstation_metrics.iloc[0]
workstation_metrics = workstation_metrics.drop(workstation_metrics.index[0])

In [103]:
def create_abs_difference(model, x, y, name="abs_diff"):
    """
    Creates variables and constraints to represent |x - y| in a PuLP model.
    
    Parameters:
    model: PuLP LpProblem
    x: PuLP variable or number representing first value
    y: PuLP variable or number representing second value
    name: Base name for the new variables (default: "abs_diff")
    
    Returns:
    abs_diff: PuLP variable representing |x - y|
    """
    # Create a variable for the absolute difference
    abs_diff = LpVariable(f"{name}", lowBound=0)
    
    # Create positive and negative part variables
    pos_part = LpVariable(f"{name}_pos", lowBound=0)
    neg_part = LpVariable(f"{name}_neg", lowBound=0)
    
    # Add constraints
    # x - y = pos_part - neg_part
    model += x - y == pos_part - neg_part, f"{name}_decomposition"
    
    # abs_diff = pos_part + neg_part
    model += abs_diff == pos_part + neg_part, f"{name}_definition"
    
    return abs_diff

In [104]:
def gb_output_coef(node_gb_per_wafer, node_yield):
    return 13 * node_gb_per_wafer * node_yield / 1e9 # in billions, hence /1e9

def calculate_gb_output(loading_dict, quarter, specs_df):
    """Calculate GB output for a given quarter's loading"""
    total_gb = 0
    for node_idx, loading in loading_dict.items():
        gb_per_wafer = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_gb_per_wafer'].values[0]
        yield_rate = specs_df[specs_df['quarter'] == quarter][f'node{node_idx}_yield'].values[0]
        total_gb += loading * gb_output_coef(gb_per_wafer, yield_rate)
    return total_gb

def optimize_loading(min_node_3_wafers=0):
    """
    Optimize loading to minimize total wafers while meeting TAM with integer constraints
    
    Args:
        min_wafers: Minimum number of wafers required per node per quarter
    """
    results = {"loading": {}}
    quarters = product_specs['quarter'].unique()
    
    # Get initial loading
    current_loading = {
        node_idx: initial_loading[f'node{node_idx}_loading'].values[0]
        for node_idx in range(1, 4)
    }
    
    results["loading"][quarters[0]] = current_loading.copy()
    
    # Optimize each quarter
    for i in range(1, len(quarters)):
        quarter = quarters[i]
        prev_quarter = quarters[i-1]
        
        prob = LpProblem(f"Profit_Maximization_{quarter}", LpMinimize)
        
        # Decision variables - specified as integers
        node1 = LpVariable(f"node1_{quarter}", 0, None, LpInteger)
        node2 = LpVariable(f"node2_{quarter}", 0, None, LpInteger)
        node3 = LpVariable(f"node3_{quarter}", min_node_3_wafers, None, LpInteger)
        
        nodes = {
            1: node1,
            2: node2,
            3: node3
        }
        
        # Target TAM
        target_tam = tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        
        # Calculate GB output coefficients
        quarter_specs = product_specs[product_specs['quarter'] == quarter]
        coefs = {
            node_idx: gb_output_coef(
                quarter_specs[f'node{node_idx}_gb_per_wafer'].values[0],
                quarter_specs[f'node{node_idx}_yield'].values[0]
            ) 
            for node_idx in nodes.keys()
        }
        
        # Constraints
        # 1. Quarter to quarter loading change constraint (±2500)
        for node_idx, node in nodes.items():
            prob += node >= results["loading"][prev_quarter][node_idx] - 2500, f"Node{node_idx}_min_change_{quarter}"
            prob += node <= results["loading"][prev_quarter][node_idx] + 2500, f"Node{node_idx}_max_change_{quarter}"
        
        # 2. TAM constraints (±2B from target)
        gb_output = sum([coefs[node_idx] * node for node_idx, node in nodes.items()])
        prob += gb_output >= target_tam - 2, f"Min_TAM_{quarter}"
        prob += gb_output <= target_tam + 2, f"Max_TAM_{quarter}"

        # create_abs_difference(prob, target_tam, gb_output, f"abs_tams_diff_{quarter}")

        # Solve
        status = prob.solve()
        
        if status != 1:
            print(f"\nWarning: Could not find optimal solution for {quarter}")
            print(f"Status code: {status}")
            print("Solver messages:")
            print(prob.solver.problem.status())
            continue
            
        results["loading"][quarter] = {
            node_idx: math.floor(node.value()) for node_idx, node in nodes.items()
        }
        
        # Print quarter results
        total_wafers = sum(results["loading"][quarter].values())
        gb_output_val = calculate_gb_output(results["loading"][quarter], quarter, product_specs)
        print(f"\n{quarter}:")
        print(f"Total wafers: {total_wafers}")
        print(f"GB output: {gb_output_val:.2f}B vs target {target_tam:.2f}B")
        for node in range(1, 4):
            print(f"{node}: {results['loading'][quarter][node]} wafers")
            if i > 0:
                change = results["loading"][quarter][node] - results["loading"][prev_quarter][node]
                print(f"  Change from prev quarter: {change:+d}")
    
    return results

def validate_solution(results):
    """Validate that solution meets all constraints with detailed output"""
    quarters = list(results['loading'].keys())
    violations = []
    
    for i in range(len(quarters)):
        quarter = quarters[i]
        
        # Check integer and non-negative loading
        for node in range(1, 4):
            if not isinstance(results['loading'][quarter][node], int):
                violations.append(f"{quarter}: {node} has non-integer loading: {results['loading'][quarter][node]}")
            if results['loading'][quarter][node] < 0:
                violations.append(f"{quarter}: {node} has negative loading: {results['loading'][quarter][node]}")
        
        # Check quarter-to-quarter change
        if i > 0:
            prev_quarter = quarters[i-1]
            for node in range(1, 4):
                change = results['loading'][quarter][node] - results['loading'][prev_quarter][node]
                if abs(change) > 2500:
                    violations.append(
                        f"{quarter}: {node} change exceeds limit: {change:+d} "
                        f"(from {results['loading'][prev_quarter][node]} to {results['loading'][quarter][node]})"
                    )
        
        # Check TAM constraints
        gb_output = calculate_gb_output(results['loading'][quarter], quarter, product_specs)
        target_tam = tam_margins[tam_margins['quarter'] == quarter]['tam'].values[0]
        if abs(gb_output - target_tam) > 2:
            violations.append(
                f"{quarter}: TAM deviation exceeds 2B: output={gb_output:.2f}B, "
                f"target={target_tam:.2f}B, diff={gb_output-target_tam:+.2f}B"
            )
    
    if violations:
        print("\nConstraint Violations:")
        for v in violations:
            print(f"- {v}")
        return False
    
    print("\nAll constraints satisfied:")
    print(f"- Integer loading values ✓")
    print(f"- Non-negative loading ✓")
    print(f"- Quarter-to-quarter changes within ±2500 ✓")
    print(f"- TAM requirements met within ±2B ✓")
    return True

In [105]:
results = optimize_loading(min_node_3_wafers = 500)
    
# Validate solution
if validate_solution(results):
    print("\nAll constraints satisfied")
else:
    print("\nSome constraints violated")

# Convert results to DataFrame for easy viewing
qn1_part_a_results_df = pd.DataFrame(results["loading"]).T
qn1_part_a_results_df.index.name = 'quarter'

qn1_part_a_results_df


Q2'26:
Total wafers: 19395
GB output: 25.40B vs target 27.40B
1: 9501 wafers
  Change from prev quarter: -2499
2: 6395 wafers
  Change from prev quarter: +1395
3: 3499 wafers
  Change from prev quarter: +2499

Q3'26:
Total wafers: 21844
GB output: 32.90B vs target 34.90B
1: 12001 wafers
  Change from prev quarter: +2500
2: 8844 wafers
  Change from prev quarter: +2449
3: 999 wafers
  Change from prev quarter: -2500

Q4'26:
Total wafers: 24481
GB output: 37.00B vs target 39.00B
1: 14501 wafers
  Change from prev quarter: +2500
2: 6481 wafers
  Change from prev quarter: -2363
3: 3499 wafers
  Change from prev quarter: +2500

Q1'27:
Total wafers: 25182
GB output: 42.70B vs target 44.70B
1: 12001 wafers
  Change from prev quarter: -2500
2: 7182 wafers
  Change from prev quarter: +701
3: 5999 wafers
  Change from prev quarter: +2500

Q2'27:
Total wafers: 24300
GB output: 49.50B vs target 51.50B
1: 9501 wafers
  Change from prev quarter: -2500
2: 6300 wafers
  Change from prev quarter: -882

,1,2,3
quarter,,,
Q1'26,12000,5000,1000
Q2'26,9501,6395,3499
Q3'26,12001,8844,999
Q4'26,14501,6481,3499
Q1'27,12001,7182,5999
Q2'27,9501,6300,8499
Q3'27,10737,8800,6000
Q4'27,8238,6300,8421


In [106]:
def tool_requirement(loading, minute_load, utilization):
    return loading * minute_load / (7 * 24 * 60 * utilization)

In [125]:
loading_profile = pd.DataFrame({
    1: [12000, 14500, 16320, 15365, 14755, 14743, 14600, 14606] ,
    2: [5000, 4887, 4096, 4362, 4530, 4567, 4024, 4469] ,
    3: [1000, 3492, 5969, 7454, 8231, 8205, 8454, 8239]
})

loading_profile.index = pd.DataFrame(results['loading']).T.index
# loading_profile = pd.DataFrame(results["loading"]).T

In [126]:
loading_profile

,1,2,3
Q1'26,12000,5000,1000
Q2'26,14500,4887,3492
Q3'26,16320,4096,5969
Q4'26,15365,4362,7454
Q1'27,14755,4530,8231
Q2'27,14743,4567,8205
Q3'27,14600,4024,8454
Q4'27,14606,4469,8239


In [127]:
# loading_profile = pd.DataFrame(results["loading"]).T

calcs = {"tools": {}, "capex": {}, "revenue": {}}

current_tool_count = dict(workstation_metrics['initial_tool_count'])
tool_costs = dict(workstation_metrics['capex_per_tool'])
current_tool_count = {ws: int(tool_count) for ws, tool_count in current_tool_count.items()}
    
calcs["tools"]["Q1'26"] = current_tool_count.copy()
calcs["capex"]["Q1'26"] = 0

calcs["revenue"]["Q1'26"] = calculate_gb_output(dict(loading_profile.loc["Q1'26"]), "Q1'26", product_specs) * 0.002 * 1e9

for quarter, nodes in loading_profile[1:].iterrows():
    tool_reqs = {}
    for ws, ws_metrics in workstation_metrics.iterrows():
        tool_requirements = [
            tool_requirement(
                node, 
                ws_metrics[f"node_{node_idx}_minute_load"], 
                ws_metrics[f"utilization"],
            ) for node_idx, node in nodes.items()
        ]
        tool_reqs[ws] = math.ceil(sum(tool_requirements))

    tool_reqs = {ws: max(tool_count, current_tool_count[ws]) for ws, tool_count in tool_reqs.items()}

    capex = sum([(tool_reqs[ws] - current_tool_count[ws]) * tool_costs[ws] for ws in current_tool_count])
    current_tool_count = tool_reqs.copy()

    revenue = calculate_gb_output(dict(nodes), quarter, product_specs) * 0.002 * 1e9
    
    calcs["tools"][quarter] = tool_reqs
    calcs["capex"][quarter] = capex
    calcs["revenue"][quarter] = revenue

In [128]:
calcs

{'tools': {"Q1'26": {'A': 10,
   'B': 18,
   'C': 5,
   'D': 11,
   'E': 15,
   'F': 2,
   'G': 23,
   'H': 3,
   'I': 4,
   'J': 1},
  "Q2'26": {'A': 12,
   'B': 24,
   'C': 8,
   'D': 13,
   'E': 16,
   'F': 4,
   'G': 33,
   'H': 4,
   'I': 4,
   'J': 2},
  "Q3'26": {'A': 14,
   'B': 30,
   'C': 10,
   'D': 13,
   'E': 16,
   'F': 6,
   'G': 42,
   'H': 4,
   'I': 4,
   'J': 3},
  "Q4'26": {'A': 14,
   'B': 32,
   'C': 10,
   'D': 13,
   'E': 16,
   'F': 7,
   'G': 44,
   'H': 4,
   'I': 4,
   'J': 3},
  "Q1'27": {'A': 14,
   'B': 33,
   'C': 11,
   'D': 13,
   'E': 16,
   'F': 7,
   'G': 44,
   'H': 4,
   'I': 4,
   'J': 3},
  "Q2'27": {'A': 14,
   'B': 33,
   'C': 11,
   'D': 13,
   'E': 16,
   'F': 7,
   'G': 44,
   'H': 4,
   'I': 4,
   'J': 3},
  "Q3'27": {'A': 14,
   'B': 33,
   'C': 11,
   'D': 13,
   'E': 16,
   'F': 7,
   'G': 44,
   'H': 4,
   'I': 4,
   'J': 3},
  "Q4'27": {'A': 14,
   'B': 33,
   'C': 11,
   'D': 13,
   'E': 16,
   'F': 7,
   'G': 44,
   'H': 4,
   'I': 

In [129]:
pd.DataFrame(calcs["tools"]).T

,A,B,C,D,E,F,G,H,I,J
Q1'26,10,18,5,11,15,2,23,3,4,1
Q2'26,12,24,8,13,16,4,33,4,4,2
Q3'26,14,30,10,13,16,6,42,4,4,3
Q4'26,14,32,10,13,16,7,44,4,4,3
Q1'27,14,33,11,13,16,7,44,4,4,3
Q2'27,14,33,11,13,16,7,44,4,4,3
Q3'27,14,33,11,13,16,7,44,4,4,3
Q4'27,14,33,11,13,16,7,44,4,4,3


In [130]:
profits = [calcs['revenue'][quarter] - calcs['capex'][quarter] for quarter in calcs['capex'].keys()]
net_profits = sum(profits)
print(f"{net_profits:,}")
print(net_profits)

455,577,893.4
455577893.4


In [131]:
pd.DataFrame(calcs["tools"]).T.to_csv("tools_part_a.csv")

PermissionError: [Errno 13] Permission denied: 'tools_part_a.csv'

In [132]:
pd.DataFrame(results["loading"]).to_csv("qn1_part_a.csv")